# Informação de Fisher

Seja $X$ uma amostra aleatória cuja distribuição depende de $\theta$ e tem valores em $(a,b) \subset \mathbb{R}$. Seja $f_n(x|\theta)$ a pdf conjunta de $X$. Assuma que $S = {x | f(x|\theta) > 0}$ é o mesmo para todo $\theta$. E $\lambda_n(x|\theta) = \log f_n(x|\theta)$ é duas vezes diferenciável em $\theta$.  A informação é: 

$$
I_n(\theta) = E_{\theta}\{[\lambda_n '(X|\theta)]^2\}
$$

Agora assuma que duas derivadas de $\int_S f_n(x|\theta)dx$ com respeito a $\theta$ podemos [inverter a ordem de integração e diferenciação](https://en.wikipedia.org/wiki/Leibniz_integral_rule#:~:text=General%20form%3A%20Differentiation%20under%20the%20integral%20sign,-Theorem.&text=That%20is%2C%20it%20is%20related,as%20the%20Leibniz%20integral%20rule.&text=the%20change%20of%20order%20of%20integration%20(integration%20under%20the%20integral,%3B%20i.e.%2C%20Fubini's%20theorem).). Então:

$$
I_n(\theta) = - E_{\theta}[\lambda_n ''(X|\theta)]
$$

## Teorema

$$I_n(\theta) = nI(\theta)$$

## Desigualdade de Cramér-Rao

Seja $X$ uma amostra aleatória com pdf $f(x| \theta)$. Suponha as hipóteses acima acerca dessa distribuição. Seja $T = r(X)$ com variância finita e $m(\theta) = E_{\theta}(T)$ é diferenciável. Assim:

$$
Var_{\theta}(T) \geq \frac{[m'(\theta)]^2}{nI(\theta)}
$$

A igualdade vale se, e somente se, existem funções $u(\theta)$ e $v(\theta)$ que podem depender em $\theta$ mas não de $X$ tal que: 

$$
T = u(\theta)\lambda_n'(X|\theta) + v(\theta)
$$

Se $T$ for não viesado $m(\theta) = \theta \implies m'(\theta) = 1$

## Estimador Eficiente

## Estimadores não viesados com variância mínima 

### Distribuição assintótica de um estimador eficiente 

### Distribuição assintótica do MLE

### Estimação Eficiente 